In [9]:
!pip install psycopg2 --quiet

In [10]:
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor

In [15]:
def connect_to_db():
    con = pg2.connect(host='postgres', 
                      dbname='postgres', 
                      user='postgres')
    cur = con.cursor(cursor_factory=RealDictCursor)
    return con, cur

def query_to_dictionary(query, fetch_res=True):
    con, cur = connect_to_db()
    cur.execute(query)
    if fetch_res:
        results = cur.fetchall()
    else:
        results = None
    con.close()
    return results

def query_to_dataframe(query):
    return DataFrame(query_to_dictionary(query))

def execute_query(query, update = False):
    HOST = 'postgres'
    DBNAME = 'postgres'
    USER = 'postgres'
    
    con = pg2.connect(host=HOST, dbname = DBNAME, user = USER)
    curs = con.cursor(cursor_factory=RealDictCursor)
    curs.execute(query)
    if not update:
        results = curs.fetchall()
        con.close()
        return results
    con.close()
    return None

In [16]:
def purge_table(table_name):
    query = '''
            BEGIN;
            DELETE FROM {};
            COMMIT;
            '''.format(table_name)
    execute_query(query, update=True)

In [17]:
con, cur = connect_to_db()

In [18]:
query = 'SELECT * FROM pages'
execute_query(query)

[{'page_id': 1, 'title': 'machine learning'}]

In [7]:
def format_data_for_query(data_list):
    new_data_list = []
    for i in data_list:
        if type(i) == str:
            new_data_list.append("'{}'".format(i))
        else:
            new_data_list.append(str(i))
    return ', '.join(new_data_list)

In [ ]:
def update_table(table_name, col_values):
    query = '''
        BEGIN;
        INSERT INTO {} VALUES ({});
        COMMIT;
        '''.format(table_name, format_data_for_query(col_values))
    query_to_dictionary(query,fetch_res=False)
    

In [ ]:
def insert_1(page_id, title, text, category):
    update_table('pages',[page_id, title])
    update_table('page_content', [page_id, text])
    update_table('page_categories', [page_id, category])

In [ ]:
insert_1(1,'machine learning','this is a fake machine learning', 'machine learning')

In [19]:
query = 'SELECT * FROM pages'

In [20]:
query_to_dictionary(query)

[{'page_id': 1, 'title': 'machine learning'}]

In [21]:
query = '''
          SELECT * FROM pages p
          JOIN page_categories c ON 
          p.page_id = c.page_id
          JOIN page_content pc ON
          p.page_id = pc.page_id
          '''
execute_query(query)

[{'category': 'machine learning',
  'content': 'this is a fake machine learning',
  'page_id': 1,
  'title': 'machine learning'}]